<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab8/sts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab8

Mounted at /content/drive
/content/drive/MyDrive/ai-intensive2
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 15.66 KiB | 9.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   c9516f9..b5d1491  main       -> origin/main
Updating c9516f9..b5d1491
Fast-forward
 lab8/token-classification.ipynb | 2154 ++++++++++++++++++++++++++++++++++++++---------------------
 1 file changed, 1405 insertions(+), 749 deletions(-)
/content/drive/MyDrive/ai-intensive2/lab8


# Representation Learning (Semantic Text Similarity)
Representation Learning이란, 주어진 입력 데이터를 임의 차원의 벡터로 바꾸는 학습 방법을 의미합니다. <br/>

이번 실습에서는, 문장 유사도 평가 데이터셋을 이용하여, 유사한 의미를 가진 두 문장을 유사한 벡터로 바꾸는 임베딩 모델을 학습해 보겠습니다. <br/>
문장 유사도는 코사인 유사도를 사용하겠습니다.

![](https://framerusercontent.com/images/ZMuuctxQ5Bhi1BK3zE1pC77hgI.png)

![](https://lh6.googleusercontent.com/ycXLRE6YUiFrVJpkYXJe8I7oCiuUSYLhfBHvn81N3_AARdiEuswYKLqC5mLNqdqTiAkfCN7hBBdrIgQi6OAbOshJE1d3q_0XuWjik_KaQqryrh63PJiS9wDwT0M0NZ_AbOFpvEZgcMjbgYBXtM3VVdM)

In [2]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 데이터셋 살펴보기
데이터셋은 문장 sentence1, sentence2와 유사도를 나타내는 0~5 사이의 실수가 있습니다(클 수록 유사함). <br/>
저희는 여기서 유사도가 3.0 이상인 경우 유사한 문장, 미만인 경우 유사하지 않은 문장으로 가정하고 유사한 문장 간에는 코사인 유사도가 커지고, 유사하지 않은 문장 간에는 코사인 유사도가 멀어지도록 학습해볼 것입니다.

In [3]:
from datasets import load_dataset

dataset = load_dataset("glue", "stsb")
dataset["train"][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

{'sentence1': 'A plane is taking off.',
 'sentence2': 'An air plane is taking off.',
 'label': 5.0,
 'idx': 0}

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### 이전 예제와 다른 점
이전 예제는 1개의 sentence가 각각 input_ids, attention_mask를 가졌지만, 이번에는 두 개의 sentence가 있습니다. 기존 data collator는 하나의 sentence만 처리하기 때문에, 두 개의 sentence를 따로 처리하도록 collator를 직접 만들어주고 DataLoader에 지정해주어야합니다. <br/>

이번 예제에서는 collate_function이 토큰화와 패딩을 모두 수행하므로, 데이터셋을 미리 토큰화할 필요가 없습니다.

In [5]:
from pprint import pprint
import torch

def collate_function(examples):
  s1 = [x["sentence1"] for x in examples]
  s2 = [x["sentence2"] for x in examples]

  # 유사도가 3.0 이상일 경우 1, 아니면 -1을 줍니다.
  labels = [1 if x["label"] >= 3.0 else -1 for x in examples]

  s1 = data_collator(tokenizer(s1, truncation=True))
  s2 = data_collator(tokenizer(s2, truncation=True))
  return {
      "sentence1": s1,
      "sentence2": s2,
      "labels": torch.tensor(labels)
  }

batch = [dataset["train"][i] for i in range(4)]
print("Original batch")
pprint(batch)

print("Processed batch")
batch = collate_function(batch)
pprint(batch)

Original batch
[{'idx': 0,
  'label': 5.0,
  'sentence1': 'A plane is taking off.',
  'sentence2': 'An air plane is taking off.'},
 {'idx': 1,
  'label': 3.799999952316284,
  'sentence1': 'A man is playing a large flute.',
  'sentence2': 'A man is playing a flute.'},
 {'idx': 2,
  'label': 3.799999952316284,
  'sentence1': 'A man is spreading shreded cheese on a pizza.',
  'sentence2': 'A man is spreading shredded cheese on an uncooked pizza.'},
 {'idx': 3,
  'label': 2.5999999046325684,
  'sentence1': 'Three men are playing chess.',
  'sentence2': 'Two men are playing chess.'}]
Processed batch
{'labels': tensor([ 1,  1,  1, -1]),
 'sentence1': {'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]),
               'input_ids': tensor([[  101,  1037,  4946,  2003,  2635,  2125,  1012,   102,     0,     0,
           

In [6]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [7]:

embeddings1 = model(**batch["sentence1"]).last_hidden_state[:, 0]
embeddings2 = model(**batch["sentence2"]).last_hidden_state[:, 0]

print(embeddings1, embeddings1.shape)
print(embeddings2, embeddings2.shape)

tensor([[-0.1035, -0.1629,  0.2163,  ..., -0.1505,  0.4066,  0.2683],
        [-0.1725, -0.0812, -0.0660,  ..., -0.0684,  0.3235,  0.1247],
        [ 0.0803, -0.1731, -0.0162,  ..., -0.1853,  0.5076,  0.1833],
        [-0.1802, -0.0538, -0.0109,  ..., -0.1360,  0.7008,  0.1090]],
       grad_fn=<SelectBackward0>) torch.Size([4, 768])
tensor([[-0.1041, -0.1481,  0.2114,  ..., -0.1327,  0.4526,  0.2184],
        [-0.1437, -0.1092, -0.1119,  ..., -0.1843,  0.3343,  0.1765],
        [-0.0225, -0.2647, -0.0403,  ..., -0.2195,  0.5004,  0.2487],
        [-0.1225, -0.0736, -0.0301,  ..., -0.1952,  0.6437,  0.0626]],
       grad_fn=<SelectBackward0>) torch.Size([4, 768])


## 아래 식을 참고하여 코사인 유사도를 계산해봅시다
![](https://studymachinelearning.com/wp-content/uploads/2019/09/Cosine-similarity-Wikipedia.png)

In [10]:
def cosine_similarity(x, y):
  ##########################
  # 코사인 유사도를 구현해봅시다
  #
  #


  dot = torch.sum(x * y, dim=-1)
  x_norm = torch.sqrt(torch.sum(x * x, dim=-1))
  y_norm = torch.sqrt(torch.sum(y * y, dim=-1))

  return dot / (x_norm * y_norm)

cos_sim = cosine_similarity(embeddings1, embeddings2)
# 정답: tensor([0.9913, 0.9926, 0.9901, 0.9948], grad_fn=<DivBackward0>)
print(cos_sim)

tensor([0.9913, 0.9926, 0.9901, 0.9948], grad_fn=<DivBackward0>)


In [11]:
cos_sim * batch["labels"]

tensor([ 0.9913,  0.9926,  0.9901, -0.9948], grad_fn=<MulBackward0>)

In [12]:
(cos_sim * batch["labels"]).mean()

tensor(0.4948, grad_fn=<MeanBackward0>)

# Training

In [13]:
import torch

batch_size = 8

# 학습을 위한 dataloader
training_loader = torch.utils.data.DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_function)
validation_loader = torch.utils.data.DataLoader(dataset["test"], batch_size=batch_size, shuffle=False, collate_fn=collate_function)


In [ ]:
next(iter(training_loader))

{'sentence1': {'input_ids': tensor([[  101,  5279,  1005,  1055,  5152, 12865, 10220,  2000,  2067,  2091,
            102,     0,     0,     0,     0],
         [  101,  1996,  2611,  2003,  4147,  1037,  2630, 17983,  3797,  1998,
          11220, 28522,  6155,  1012,   102],
         [  101,  1037,  2450,  2003, 16475, 12377,  2075, 20130,  1012,   102,
              0,     0,     0,     0,     0],
         [  101,  1996,  2611, 10948,  1999,  2392,  1997,  1996, 28698,  1012,
            102,     0,     0,     0,     0],
         [  101,  2610, 12210,  2129,  8343,  2001,  4110,   102,     0,     0,
              0,     0,     0,     0,     0],
         [  101,  1037,  2450,  2003,  5559,  2006,  1037,  3586,  1012,   102,
              0,     0,     0,     0,     0],
         [  101,  1045,  2123,  1005,  1056,  3305,  2129,  4071,  1997,  4613,
           2097,  2022, 10890,  1012,   102],
         [  101, 10047,  5521,  7499,  2015,  6538,  2005, 11729,  4028,   102,
           

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm


def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

def train_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.
    log_freq = 50

    for i, batch in enumerate(tqdm(training_loader, desc=f"Epoch {epoch_index}")):
        optimizer.zero_grad()
        sentence1, sentence2, labels = batch["sentence1"], batch["sentence2"], batch["labels"]

        embeddings1 = model(**change_dict_device(sentence1)).last_hidden_state[:, 0]
        embeddings2 = model(**change_dict_device(sentence2)).last_hidden_state[:, 0]

        cos_sim = cosine_similarity(embeddings1, embeddings2)

        # 유사도에 label을 곱한 값의 음수를 loss로 씁니다.
        # 유사한 두 쌍(label=1)의 유사도는 더 큰 값이 되도록 학습되고
        # 유사하지 않은 두 쌍의 유사도는 더 낮은 값이 되도록 학습됩니다.
        loss = -(cos_sim * labels.to(device)).mean()

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % log_freq == 0:
            last_loss = running_loss / log_freq
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch))

    model.train(True)
    avg_loss = train_epoch(epoch)

    print(f"LOSS train {avg_loss}")

EPOCH 0:


Epoch 0:   7%|▋         | 52/719 [00:06<00:50, 13.19it/s]

  batch 50 loss: -0.14250615552067758


Epoch 0:  14%|█▍        | 102/719 [00:10<00:42, 14.58it/s]

  batch 100 loss: -0.16302502557635307


...

  batch 650 loss: -0.23657177165150642


Epoch 0:  98%|█████████▊| 702/719 [00:52<00:01, 14.85it/s]

  batch 700 loss: -0.23117996118962764


Epoch 0: 100%|██████████| 719/719 [00:53<00:00, 13.33it/s]

LOSS train -0.23117996118962764


# 사용해보기

In [ ]:
model.eval()

test_batch = [dataset["validation"][i + 4] for i in range(4)]
pprint(test_batch)
test_batch = collate_function(test_batch)


sentence1, sentence2, labels = test_batch["sentence1"], test_batch["sentence2"], test_batch["labels"]

with torch.no_grad():
  embeddings1 = model(**change_dict_device(sentence1)).last_hidden_state[:, 0]
  embeddings2 = model(**change_dict_device(sentence2)).last_hidden_state[:, 0]

  pprint(embeddings1)
  pprint(embeddings2)
  cos_sim = cosine_similarity(embeddings1, embeddings2)
  print(cos_sim)

[{'idx': 4,
  'label': 2.75,
  'sentence1': 'A woman is playing the flute.',
  'sentence2': 'A man is playing a flute.'},
 {'idx': 5,
  'label': 2.615000009536743,
  'sentence1': 'A woman is cutting an onion.',
  'sentence2': 'A man is cutting onions.'},
 {'idx': 6,
  'label': 5.0,
  'sentence1': 'A man is erasing a chalk board.',
  'sentence2': 'The man is erasing the chalk board.'},
 {'idx': 7,
  'label': 2.3329999446868896,
  'sentence1': 'A woman is carrying a boy.',
  'sentence2': 'A woman is carrying her baby.'}]
tensor([[-0.6556,  1.2672,  0.8618,  ..., -0.3208, -0.5108,  1.2752],
        [ 0.2453,  0.1759, -0.0490,  ..., -0.3517, -0.4729,  0.0221],
        [-0.0942,  0.2463, -0.6775,  ...,  0.7738,  0.1720,  0.7072],
        [ 0.2514, -0.4294, -0.4119,  ..., -0.1878, -0.2837,  0.3989]],
       device='cuda:0')
tensor([[-0.7387,  1.3417,  0.8741,  ..., -0.3217, -0.5142,  1.2607],
        [ 0.5253,  0.0581, -0.2301,  ..., -0.4841, -0.1188, -0.4095],
        [ 0.0165,  0.1745, -0.

# Contrasive Learning with In-batch negative

Representation Learning에서 빠지기 쉬운 함정 중 하나는 모든 임베딩 값을 0으로 만들어버리는 현상입니다. 하지만 유사하지 않은 negative 쌍에서 유사도를 멀어지게 만들 경우에는 이런 현상을 피할 수 있습니다. 이를 contrasive learning 이라고 합니다.

![](https://blog.kakaocdn.net/dn/emIyBi/btqUOCkyTQH/lYbz0OJNM7Pa6rRK5lKVTK/img.png)

하지만 negative가 없는 경우가 있습니다. 예를들어 이미지-캡션 쌍이 대표적인 예시입니다. 이 경우에는 한 배치 내의 다른 instance를 negative로 사용하는 In-batch negative 방법을 이용합니다.
![](https://lilianweng.github.io/posts/2021-05-31-contrastive/CLIP.png)




In [ ]:
# negative case를 모두 제거하고 positive만 남깁니다.
cl_dataset = dataset.filter(lambda x: x["label"] >= 3.0)
cl_dataset['train'][:4]

Filter:   0%|          | 0/5749 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1379 [00:00<?, ? examples/s]

{'sentence1': ['A plane is taking off.',
  'A man is playing a large flute.',
  'A man is spreading shreded cheese on a pizza.',
  'A man is playing the cello.'],
 'sentence2': ['An air plane is taking off.',
  'A man is playing a flute.',
  'A man is spreading shredded cheese on an uncooked pizza.',
  'A man seated is playing the cello.'],
 'label': [5.0, 3.799999952316284, 3.799999952316284, 4.25],
 'idx': [0, 1, 2, 4]}

In [ ]:
model = AutoModel.from_pretrained("distilbert-base-uncased")

### 벡터간의 코사인 유사도를 빠르게 구하기
![](https://pytorch.org/assets/images/inside-the-matrix/matmul3.jpg)

전치행렬과 행렬곱을 수행하면 모든 벡터-벡터 쌍의 정규화되지 않은 코사인 유사도를 구할 수 있습니다.

In [ ]:
batch = [dataset["train"][i] for i in range(4)]
batch = collate_function(batch)

embeddings1 = model(**batch["sentence1"]).last_hidden_state[:, 0]
embeddings2 = model(**batch["sentence2"]).last_hidden_state[:, 0]

cos_matrix = (embeddings1 @ embeddings2.T).softmax(-1)
pprint(cos_matrix)

# torch.diag 함수는 대각요소만 가져옵니다.
pprint(torch.diag(cos_matrix))

pprint(-torch.diag(cos_matrix).log().mean())

tensor([[9.9966e-01, 4.4058e-08, 6.3955e-05, 2.7834e-04],
        [1.9339e-03, 1.2293e-01, 8.7195e-01, 3.1825e-03],
        [7.1816e-06, 2.1026e-07, 9.9998e-01, 1.3685e-05],
        [3.3759e-03, 1.8165e-07, 3.4494e-04, 9.9628e-01]],
       grad_fn=<SoftmaxBackward0>)
tensor([0.9997, 0.1229, 1.0000, 0.9963], grad_fn=<DiagonalBackward0_copy>)
tensor(0.5250, grad_fn=<NegBackward0>)


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm


def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

def train_epoch_contrasive(epoch_index):
    running_loss = 0.
    last_loss = 0.
    log_freq = 50

    for i, batch in enumerate(tqdm(training_loader, desc=f"Epoch {epoch_index}")):
        optimizer.zero_grad()
        sentence1, sentence2 = batch["sentence1"], batch["sentence2"]

        embeddings1 = model(**change_dict_device(sentence1)).last_hidden_state[:, 0]
        embeddings2 = model(**change_dict_device(sentence2)).last_hidden_state[:, 0]

        cos_matrix = (embeddings1 @ embeddings2.T).softmax(-1)
        loss = -torch.diag(cos_matrix).log().mean()
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % log_freq == 0:
            last_loss = running_loss / log_freq
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch))

    model.train(True)
    avg_loss = train_epoch_contrasive(epoch)

    print(f"LOSS train {avg_loss}")

EPOCH 0:


Epoch 0:   7%|▋         | 52/719 [00:03<00:43, 15.42it/s]

  batch 50 loss: 1.0414105576389492


Epoch 0:  14%|█▍        | 102/719 [00:07<00:49, 12.57it/s]

  batch 100 loss: 0.24518281796947122


...

  batch 650 loss: 0.10192377474915702


Epoch 0:  98%|█████████▊| 702/719 [00:50<00:01, 14.88it/s]

  batch 700 loss: 0.2856637429469265


Epoch 0: 100%|██████████| 719/719 [00:52<00:00, 13.80it/s]

LOSS train 0.2856637429469265
